In [12]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.3.1'
spark_version = 'spark-3.3.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease
Get:4 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:5 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:9 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [2,776 kB]
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:11 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [3,255 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:13 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1,354 kB]


In [13]:
# Import findspark and initialize. 
import findspark
findspark.init()

In [14]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [20]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)

#Show DataFrame
df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [80]:
# 2. Create a temporary view of the DataFrame.

df.createOrReplaceTempView('home_sales')

In [121]:
from IPython.core.history import datetime
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
average_price_df = spark.sql("""
SELECT 
YEAR(date) AS YEAR, 
ROUND(AVG(price), 2) AS AVERAGE_PRICE
FROM home_sales
Where Bedrooms = 4
GROUP BY YEAR
ORDER BY YEAR
""")
average_price_df.show()

+----+-------------+
|YEAR|AVERAGE_PRICE|
+----+-------------+
|2019|     300263.7|
|2020|    298353.78|
|2021|    301819.44|
|2022|    296363.88|
+----+-------------+



In [125]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
average_home_price_df = """
SELECT
  date_built AS YEAR,
  ROUND(AVG(price), 2) AS AVERAGE_PRICE
FROM home_sales
WHERE bedrooms = 3
  AND bathrooms = 3
GROUP BY date_built
ORDER BY date_built DESC
"""
spark.sql(average_home_price_df).show()








+----+-------------+
|YEAR|AVERAGE_PRICE|
+----+-------------+
|2017|    292676.79|
|2016|    290555.07|
|2015|     288770.3|
|2014|    290852.27|
|2013|    295962.27|
|2012|    293683.19|
|2011|    291117.47|
|2010|    292859.62|
+----+-------------+



In [133]:
#  5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
sqft_df = """
SELECT
  YEAR(date_built) AS YEAR,
  ROUND(AVG(price), 2) AS AVERAGE_PRICE
FROM home_sales
WHERE bedrooms = 3
  AND bathrooms = 3
  AND sqft_living >= 2000
  AND floors = 2
GROUP BY date_built
ORDER BY date_built DESC
"""
spark.sql(sqft_df).show()







AnalysisException: ignored

In [84]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()
view_rating= spark.sql("""
SELECT
view,
ROUND(AVG(price), 2) AS AVERAGE_PRICE
FROM home_sales
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY view
""")
view_rating.show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|AVERAGE_PRICE|
+----+-------------+
|  51|    788128.21|
|  52|    733780.26|
|  53|     755214.8|
|  54|    798684.82|
|  55|    771153.32|
|  56|     718176.4|
|  57|     734340.5|
|  58|    759764.65|
|  59|     791453.0|
|  60|    754939.65|
|  61|    746877.59|
|  62|    759150.14|
|  63|    711614.55|
|  64|    767036.67|
|  65|    736679.93|
|  66|     712475.0|
|  67|    737970.96|
|  68|    716785.44|
|  69|    750537.94|
|  70|    695865.58|
+----+-------------+
only showing top 20 rows

--- 0.6735608577728271 seconds ---


In [85]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [86]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [99]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()
runtime_df = spark.sql("""
SELECT
view,
ROUND(AVG(price), 2) AS AVERAGE_PRICE
FROM home_sales
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY view
""")

runtime_df.show()


print("--- %s seconds ---" % (time.time() - start_time))


+----+-------------+
|view|AVERAGE_PRICE|
+----+-------------+
|  51|    788128.21|
|  52|    733780.26|
|  53|     755214.8|
|  54|    798684.82|
|  55|    771153.32|
|  56|     718176.4|
|  57|     734340.5|
|  58|    759764.65|
|  59|     791453.0|
|  60|    754939.65|
|  61|    746877.59|
|  62|    759150.14|
|  63|    711614.55|
|  64|    767036.67|
|  65|    736679.93|
|  66|     712475.0|
|  67|    737970.96|
|  68|    716785.44|
|  69|    750537.94|
|  70|    695865.58|
+----+-------------+
only showing top 20 rows

--- 0.7364628314971924 seconds ---


In [92]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
df.write.partitionBy('date_built').parquet('p_home_sales', mode='overwrite')

In [93]:
# 11. Read the formatted parquet data.
p_homes_df =spark.read.parquet('p_home_sales')

In [94]:
# 12. Create a temporary table for the parquet data.
spark.sql('uncache table home_sales')

DataFrame[]

In [95]:
# 13. Run the query that filters out the view ratings with average price of greater than or eqaul to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version. 

start_time = time.time()
view_rating_df=spark.sql("""
SELECT
view,
ROUND(AVG(price), 2) AS AVERAGE_PRICE
FROM home_sales
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY view
""")

view_rating_df.show()
print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|AVERAGE_PRICE|
+----+-------------+
|  51|    788128.21|
|  52|    733780.26|
|  53|     755214.8|
|  54|    798684.82|
|  55|    771153.32|
|  56|     718176.4|
|  57|     734340.5|
|  58|    759764.65|
|  59|     791453.0|
|  60|    754939.65|
|  61|    746877.59|
|  62|    759150.14|
|  63|    711614.55|
|  64|    767036.67|
|  65|    736679.93|
|  66|     712475.0|
|  67|    737970.96|
|  68|    716785.44|
|  69|    750537.94|
|  70|    695865.58|
+----+-------------+
only showing top 20 rows

--- 0.8613502979278564 seconds ---


In [96]:
# 14. Uncache the home_sales temporary table.
spark.sql('uncache table home_sales')

DataFrame[]

In [97]:
# 15. Check if the home_sales is no longer cached
if spark.catalog.isCached('home_sales'):
  print("Cached")
else:
  print('Not Cached')


Not Cached
